In [2]:
import numpy as np
import json
import psycopg2

## Loading data into memory:

In [ ]:
recipes = None 
ingredients = None 

with np.load('simplified-recipes-1M.npz', allow_pickle=True) as data:
    recipes = data['recipes']
    ingredients = data['ingredients']

## Get unique grocery items:

In [ ]:
grocery_items = set()
LIMIT = 100

In [ ]:
for recipe in recipes[:LIMIT]:
    for ingredient in ingredients[recipe]:
        if ingredient not in grocery_items:
            grocery_items.add(ingredient)

In [ ]:
grocery_items.remove('a')
grocery_items.remove('ice')
grocery_items.remove('dry')
grocery_items.remove('any')
grocery_items.remove('more')
grocery_items.remove('plus')
grocery_items.remove('yellow')
grocery_items.remove('fat free')
grocery_items.remove('american')
grocery_items.remove('hot water')
grocery_items.remove('cold milk')
grocery_items.remove('large eggs')          
grocery_items.remove('boiling water')
grocery_items.remove('potatoes')
grocery_items.remove('egg whites')
grocery_items.remove('egg white')
grocery_items.remove('egg yolks')
grocery_items.remove('egg yolk')
grocery_items.remove('white')
grocery_items.remove('yolk')
grocery_items.remove('ground')
grocery_items.remove('heavy cream')
grocery_items.remove('ground nutmeg')
grocery_items.remove('chopped garlic')
grocery_items.remove('fresh lemon')
grocery_items.remove('unsalted butter')
grocery_items.remove('crust')
grocery_items.remove('packed')
grocery_items.remove('carrots')
grocery_items.remove('black')
grocery_items.remove('nuts')
grocery_items.remove('kosher')
grocery_items.remove('vegetable')

grocery_items.add('chestnuts')

In [ ]:
to_remove = set()

for upper_element in grocery_items:
    for lower_element in grocery_items:
        if upper_element == lower_element:
            continue
        elif upper_element in lower_element:
            to_remove.add(lower_element)
            
grocery_items = grocery_items.difference(to_remove)

In [ ]:
# sorted(grocery_items, key=len)

In [ ]:
temp = list(grocery_items)
grocery_items = {}
grocery_items_mapping = {}

for i in range(len(temp)):
    grocery_items[i]=temp[i]
    grocery_items_mapping[temp[i]]=i
# grocery_items

## Associate recipes with the ingredients they use 

In [ ]:
LIMIT = 50

with open('recipes/recipes_raw_nosource_epi.json') as data:
    all_recipes = json.load(data)
    keys = list(all_recipes)
    recipes = []
    
    i = 0
    while i < LIMIT:
        recipes.append(all_recipes[keys[i]])
        i = i + 1

In [ ]:
items = {x[1] for x in grocery_items.items()} 

In [ ]:
for i in range(len(recipes)):
    ingredient_list = set()
    for ingredients in recipes[i]['ingredients']:
        for element in grocery_items.items():
            if element[1] not in ingredient_list:
                if element[1] in ingredients:
                    ingredient_list.add(element[1])
    recipes[i]['ingredient_list']=ingredient_list

In [ ]:
# for i in range(len(recipes)):
#     print(recipes[i]['ingredient_list'])

In [ ]:
# 'chestnuts' in [x[1] for x in grocery_items.items() ]

In [ ]:
for i in range(len(recipes)):
    recipes[i]['ingredient_list'] = { grocery_items_mapping[x] for x in recipes[i]['ingredient_list'] }

In [ ]:
recipes[1]

## Set up the Database

#### Establish Connection

In [3]:
HOST = ''
DATABASE = ''
USER = ''
PASSWORD = ''
PORT = ''

with open('config.json') as data:
    configuration = json.load(data)
    HOST = configuration['HOST']
    DATABASE = configuration['DATABASE']
    USER = configuration['USER']
    PASSWORD = configuration['PASSWORD']
    PORT = configuration['PORT']

In [4]:
conn = psycopg2.connect( 
    host=HOST,
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    port=PORT)

In [9]:
cur = conn.cursor()

### Create tables

In [ ]:
# commands = ( 
#     """
#     CREATE TABLE recipes (
#         recipe_id SERIAL PRIMARY KEY,
#         title VARCHAR(255) NOT NULL,
#         full_ingredients VARCHAR [] NOT NULL,
#         instructions VARCHAR NOT NULL
#     )
#     """,
#     """
#     CREATE TABLE ingredients (
#         ingredient_id SERIAL PRIMARY KEY,
#         name VARCHAR(255) NOT NULL
#     )
#     """,
#     """
#     CREATE TABLE recipe_ingredients(
#         riid SERIAL PRIMARY KEY,
#         rid INTEGER NOT NULL,
#         iid INTEGER NOT NULL,
#         FOREIGN KEY (rid) REFERENCES recipes (recipe_id) 
#         ON UPDATE CASCADE ON DELETE CASCADE,
#         FOREIGN KEY (iid) REFERENCES ingredients (ingredient_id)
#         ON UPDATE CASCADE ON DELETE CASCADE
#     )
#     """ )

In [ ]:
# for command in commands:
#     cur.execute(command)

### Populate tables

#### Ingredients

In [ ]:
# ingredient_names = "'"+"','".join([x[1] for x in grocery_items.items()])+"'"

In [180]:
# commands = []
# for ingredient_id in grocery_items.keys():
#     ingredient_name = grocery_items[ingredient_id]
#     command = f"INSERT INTO ingredients(ingredient_id,name) VALUES ({ingredient_id},'{ingredient_name}')"
#     commands.append(command)

# commands = tuple(commands)

In [181]:
# for command in commands:
#     cur.execute(command)

#### Recipes

In [184]:
# commands = []
# for recipe_num in range(len(recipes)):
#     title = recipes[recipe_num]['title'].replace("'",'').strip()
#     full_ingredients = ','.join(recipes[recipe_num]['ingredients']).replace("'",'').strip()
#     full_ingredients = "'{"+full_ingredients+"}'"
#     instructions = recipes[recipe_num]['instructions'].replace("'",'').replace("\n",' ').strip()    
#     command = f"INSERT INTO recipes(recipe_id, title, full_ingredients, instructions) VALUES ({recipe_num}, '{title}', {full_ingredients}, '{instructions}')"
#     commands.append(command)

# commands = tuple(commands)

In [ ]:
# for command in commands:
#     cur.execute(command)

#### Recipe-Ingredient

In [187]:
# commands = []
# i = 0
# for recipe_num in range(len(recipes)):
#     for ingredient_num in recipes[recipe_num]['ingredient_list']:
#         i = i + 1
#         command = f"INSERT INTO recipe_ingredients(riid, rid, iid) VALUES ({i},{recipe_num},{ingredient_num})"
#         commands.append(command)
        
# commands = tuple(commands)

In [ ]:
# for command in commands:
#     print(command)
#     cur.execute(command)

### Cleanup

In [ ]:
cur.execute('SELECT * from recipe_ingredients')
cur.fetchall()

In [190]:
conn.commit()

In [8]:
cur.close()

In [11]:
conn.close()

# Further SQL code

In [ ]:
"""
create table fridge
(
	fridge_id serial,
	user_id int not null
		constraint fridge_users_user_id_fk
			references users
				on update cascade on delete cascade,
	ingredient_id int
		constraint fridge_ingredients_ingredient_id_fk
			references ingredients
				on update cascade on delete cascade,
	quantity int,

	constraint unique_user_ingredient unique(user_id,ingredient_id)
);

create unique index fridge_fridge_id_uindex
	on fridge (fridge_id);

alter table fridge
	add constraint fridge_pk
		primary key (fridge_id);"""

In [12]:
HOST = ''
DATABASE = ''
USER = ''
PASSWORD = ''
PORT = ''

with open('config.json') as data:
    configuration = json.load(data)
    HOST = configuration['HOST']
    DATABASE = configuration['DATABASE']
    USER = configuration['USER']
    PASSWORD = configuration['PASSWORD']
    PORT = configuration['PORT']

In [13]:
conn = psycopg2.connect( 
    host=HOST,
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    port=PORT)

In [15]:
cur = conn.cursor()

In [23]:
cur.execute('SELECT user_id from users')
userIds = [x[0] for x in cur.fetchall()]

In [ ]:
cur.execute('SELECT ingredient_id FROM ingredients')
ingredientIds = [x[0] for x in cur.fetchall()]

In [ ]:
for uid in userIds:
    for iid in ingredientIds:
        command = f"INSERT INTO fridge(user_id,ingredient_id,quantity) VALUES({uid},{iid},0)"
        cur.execute(command)

In [ ]:
conn.commit()

In [8]:
cur.close()

In [11]:
conn.close()